# 作业一：语言模型 Part2

* 书接上回，上一周实现了一个针对单句的语言模型，本周将在上周模型的基础上进一步探索
* 本周的尝试主要分为两个部分
    1. 采用BinaryLogLoss+负例采样
    2. 使用额外的上下文信息进行训练

**首先还是导入这次作业需要的包，并设置随机种子**

In [1]:
import datetime
import random
from collections import Counter, defaultdict
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_random_seed(2020)
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

**设定计算设备与数据集路径**

In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
data_path = Path('/media/bnu/data/nlp-practice/language-model')

print('PyTorch Version:', torch.__version__)
print('-' * 60)
if torch.cuda.is_available():
    print('CUDA Device Count:', torch.cuda.device_count())
    print('CUDA Device Name:')
    for i in range(torch.cuda.device_count()):
        print('\t', torch.cuda.get_device_name(i))
    print('CUDA Current Device Index:', torch.cuda.current_device())
    print('-' * 60)
print('Data Path:', data_path)

PyTorch Version: 1.4.0
------------------------------------------------------------
CUDA Device Count: 2
CUDA Device Name:
	 GeForce RTX 2080 Ti
	 GeForce RTX 2080 Ti
CUDA Current Device Index: 0
------------------------------------------------------------
Data Path: /media/bnu/data/nlp-practice/language-model


# BinaryLogLoss+负例采样

## 数据处理

### 定义单词表类

* 定义`Vocab`类用于存储单词表
* `Vocab`类当中包含了单词(token)与索引(index)之间的映射

In [3]:
class Vocab:
    def __init__(self, vocab_path):
        self.stoi = {}  # token -> index (dict)
        self.itos = []  # index -> token (list)
        
        with open(vocab_path) as f:
            # bobsue.voc.txt中，每一行是一个单词
            for w in f.readlines():
                w = w.strip()
                if w not in self.stoi:
                    self.stoi[w] = len(self.itos)
                    self.itos.append(w)

    def __len__(self):
        return len(self.itos)

**简单测试**

In [5]:
vocab = Vocab(data_path / 'bobsue.voc.txt')
print('单词表大小：', len(vocab))
print('-' * 60)
print('样例（单词 -> 索引）：')
print(list(vocab.stoi.items())[:5])
print('-' * 60)
print('样例（索引 -> 单词）：')
print(list(enumerate(vocab.itos))[:5])

单词表大小： 1498
------------------------------------------------------------
样例（单词 -> 索引）：
[('<s>', 0), ('</s>', 1), ('.', 2), ('to', 3), ('Bob', 4)]
------------------------------------------------------------
样例（索引 -> 单词）：
[(0, '<s>'), (1, '</s>'), (2, '.'), (3, 'to'), (4, 'Bob')]


### 定义语料库

* 定义`Corpus`类读取训练集、验证集、测试集语料
* 语料文件中每一行都是一个句子，也就是我们训练时的一份样本
* 将语料中的句子读入后，根据`Vocab`转换成索引列表
* `Corpus`类当中包含了语料库中的单词的计数信息与词频信息

In [8]:
class Corpus:
    def __init__(self, data_path, sort_by_len=False, 
                 uniform=False, freq_coef=0.75):
        self.vocab = Vocab(data_path / 'bobsue.voc.txt')
        self.sort_by_len = sort_by_len
        self.train_data = self.tokenize(data_path / 'bobsue.lm.train.txt')
        self.valid_data = self.tokenize(data_path / 'bobsue.lm.dev.txt')
        self.test_data = self.tokenize(data_path / 'bobsue.lm.test.txt')

        # 统计训练集的单词计数
        self.word_counter = Counter()
        for x in self.train_data:
            # 注意<s>不在我们的预测范围内，不要统计
            self.word_counter += Counter(x[1:])
        # 训练集中需要预测的总词数
        total_words = len(list(self.word_counter.elements()))

        if uniform:  # 均匀分布
            self.word_freqs = np.array(
                [0.] + [1. for _ in range(len(self.vocab) - 1)],
                dtype=np.float32
            )
            self.word_freqs = self.word_freqs / sum(self.word_freqs)
        else:  # 词频分布（提升freq_coef次方）
            self.word_freqs = np.array(
                [self.word_counter[i] for i in range(len(self.vocab))],
                dtype=np.float32
            )
            self.word_freqs = self.word_freqs / sum(self.word_freqs)
            self.word_freqs = self.word_freqs ** freq_coef
            self.word_freqs = self.word_freqs / sum(self.word_freqs)
        
    def tokenize(self, text_path):
        with open(text_path) as f:
            index_data = []  # 索引数据，存储每个样本的单词索引列表
            for s in f.readlines():
                index_data.append(
                    self.sentence_to_index(s)
                )
        if self.sort_by_len:  # 为了提升训练速度，可以考虑将样本按照长度排序，这样可以减少padding
            index_data = sorted(index_data, key=lambda x: len(x), reverse=True)
        return index_data
    
    def sentence_to_index(self, s):
        return [self.vocab.stoi[w] for w in s.split()]
    
    def index_to_sentence(self, x):
        return ' '.join([self.vocab.itos[i] for i in x])

**简单测试**

In [9]:
corpus = Corpus(data_path, sort_by_len=False)
print('训练集句子数目：', len(corpus.train_data))
print('验证集句子数目：', len(corpus.valid_data))
print('测试集句子数目：', len(corpus.test_data))
print('-' * 60)
print('训练集总共单词数目：', sum([len(x) for x in corpus.train_data]))
print('验证集总共单词数目：', sum([len(x) for x in corpus.valid_data]))
print('测试集总共单词数目：', sum([len(x) for x in corpus.test_data]))
print('-' * 60)
print('训练集预测单词数目：', sum([len(x) - 1 for x in corpus.train_data]))
print('验证集预测单词数目：', sum([len(x) - 1 for x in corpus.valid_data]))
print('测试集预测单词数目：', sum([len(x) - 1 for x in corpus.test_data])) 
print('-' * 60)
print('数据样本：')
for i in range(5):
    print(corpus.train_data[i])
    print(corpus.index_to_sentence(corpus.train_data[i]))
print('-' * 60)
print()

corpus = Corpus(data_path, sort_by_len=False, uniform=True)
print('均匀分布：')
print('词频样本：', corpus.word_freqs[:5])
print('词频个数：', len(corpus.word_freqs))

corpus = Corpus(data_path, sort_by_len=False, uniform=False, freq_coef=0.75)
print('词频分布：')
print('词频样本：', corpus.word_freqs[:5])
print('词频个数：', len(corpus.word_freqs))

训练集句子数目： 6036
验证集句子数目： 750
测试集句子数目： 750
------------------------------------------------------------
训练集总共单词数目： 71367
验证集总共单词数目： 8707
测试集总共单词数目： 8809
------------------------------------------------------------
训练集预测单词数目： 65331
验证集预测单词数目： 7957
测试集预测单词数目： 8059
------------------------------------------------------------
数据样本：
[0, 16, 235, 372, 10, 60, 3, 75, 618, 39, 2, 1]
<s> She ate quickly and asked to be taken home . </s>
[0, 38, 192, 222, 32, 31, 4, 2, 1]
<s> The girl broke up with Bob . </s>
[0, 7, 842, 2, 1]
<s> Sue apologized . </s>
[0, 12, 150, 18, 8, 261, 3, 546, 102, 5, 1097, 2, 1]
<s> He tried for a year to break into the market . </s>
[0, 200, 706, 14, 5, 26, 15, 427, 228, 2, 1]
<s> So far , the day had gone well . </s>
------------------------------------------------------------

均匀分布：
词频样本： [0.       0.000668 0.000668 0.000668 0.000668]
词频个数： 1498
词频分布：
词频样本： [0.         0.03825217 0.03720167 0.01996209 0.01417771]
词频个数： 1498


### 定义语言模型负例采样DataSet

* 这里使用PyTorch中的`DataSet`来构建我们自己的语言模型数据集
* 我们自定义的类继承`DataSet`后，要实现`__len__`与`__getitem__`方法
* 每个样本的输入是前n-1个单词，正例为后n-1个单词，负例根据词频和设定的生产个数进行生成

In [10]:
class BobSueNegSampleDataSet(torch.utils.data.Dataset):
    
    def __init__(self, index_data, word_freqs, n_negs=20):
        self.index_data = index_data  # 转换为序号的文本
        self.n_negs = n_negs  # 生成负例个数
        self.word_freqs = torch.FloatTensor(word_freqs)  # 词频
        
    def __getitem__(self, i):
        inputs = torch.LongTensor(self.index_data[i][:-1])
        poss = torch.LongTensor(self.index_data[i][1:])
        
        # 生成n_negs个负例
        negs = torch.zeros((len(poss), self.n_negs), dtype=torch.long)
        for i in range(len(poss)):
            negs[i] = torch.multinomial(self.word_freqs, self.n_negs)        
        
        return inputs, poss, negs
        
    def __len__(self):
        return len(self.index_data)

**简单测试**

In [11]:
corpus = Corpus(data_path, sort_by_len=False, uniform=False, freq_coef=0.75)
train_set = BobSueNegSampleDataSet(corpus.train_data, corpus.word_freqs)
print('训练集大小：', len(train_set))
print()
print('训练集样本：')
print('输入大小：', train_set[10][0].shape)
print('正例大小：', train_set[10][1].shape)
print('负例大小：', train_set[10][2].shape)

训练集大小： 6036

训练集样本：
输入大小： torch.Size([12])
正例大小： torch.Size([12])
负例大小： torch.Size([12, 20])


### 定义语言模型的DataLoader

* 这部分跟Part1相同，需要自定义collate_fn来处理这个问题

In [41]:
def neglm_collate_fn(batch):
    # 首先将batch的格式进行转换
    # batch[0]：Inputs
    # batch[1]: Poss
    # batch[2]: Negs
    batch = list(zip(*batch))
    
    # lengths: (batch_size)
    lengths = torch.LongTensor([len(x) for x in batch[0]])
    # inputs: (batch_size, max_len)
    inputs = nn.utils.rnn.pad_sequence(batch[0], batch_first=True)
    # poss: (batch_size, max_len)
    poss = nn.utils.rnn.pad_sequence(batch[1], batch_first=True)
    # negs: (batch_size, max_len, n_negs)
    negs = nn.utils.rnn.pad_sequence(batch[2], batch_first=True)
    # mask: (batch_size, max_len)
    mask = (poss != 0).float()
    
    return inputs, poss, negs, lengths, mask

**简单测试**

In [45]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=8,
    shuffle=True,
    collate_fn=neglm_collate_fn
)

inputs, poss, negs, lengths, mask = next(iter(train_loader))
print('Input Shape:', inputs.shape)
print('Poss Shape:', poss.shape)
print('Negs Shape:', negs.shape)
print('-' * 60)
print('Lengths:')
print(lengths)
print('Mask:')
print(mask)

Input Shape: torch.Size([8, 16])
Poss Shape: torch.Size([8, 16])
Negs Shape: torch.Size([8, 16, 20])
------------------------------------------------------------
Lengths:
tensor([12, 12,  9, 14,  9, 16,  9, 12])
Mask:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]])


## 定义网络结构

* 这里实现一个基于LSTM的网络架构，Embedding层维度与LSTM隐含层维度相同
* Inputs数据使用一个Embedding层，Postive和Negtive使用另一个Embedding层
* 损失函数与Word2Vec中的负例采样损失相同

In [50]:
class NegSampleLM(nn.Module):
    
    def __init__(self, n_words, n_embed=200, dropout=0.5):
        super(NegSampleLM, self).__init__()
        self.drop = nn.Dropout(0.5)
        # 输入的Embedding
        self.embed_in = nn.Embedding(n_words, n_embed)
        # 输出的Embedding
        self.embed_out = nn.Embedding(n_words, n_embed)
        # 这里embed_size一定要和hidden_size相同，为了之后点积计算loss
        self.lstm = nn.LSTM(n_embed, n_embed, batch_first=True)
        
    
    def forward(self, inputs, poss, negs, lengths, mask):
        # x_embed: (batch_size, seq_len, embed_size)
        x_embed = self.drop(self.embed_in(inputs))
        # poss_embed: (batch_size, seq_len, embed_size)
        poss_embed = self.embed_out(poss)
        # negs_embed: (batch_size, seq_len, n_negs, embed_size)
        negs_embed = self.embed_out(negs)
        
        x_embed = nn.utils.rnn.pack_padded_sequence(
            x_embed, lengths, batch_first=True, enforce_sorted=False
        )
        # x_lstm: (batch_size, seq_len, embed_size)
        x_lstm, _ = self.lstm(x_embed)        
        x_lstm, _ = nn.utils.rnn.pad_packed_sequence(
            x_lstm, batch_first=True
        )
        
        # x_lstm: (batch_size * seq_len, embed_size, 1)
        x_lstm = x_lstm.view(-1, x_lstm.shape[2], 1)

        # poss_embed: (batch_size * seq_len, 1, embed_size)
        poss_embed = poss_embed.view(-1, 1, poss_embed.shape[2])
        # negs_embed: (batch_size * seq_len, n_negs, embeds)
        negs_embed = negs_embed.view(-1, negs_embed.shape[2], negs_embed.shape[3])
        
        # poss_mm: (batch_size * seq_len)
        poss_mm = torch.bmm(poss_embed, x_lstm).squeeze()
        # negs_mm: (batch_size * seq_len, n_negs)
        negs_mm = torch.bmm(negs_embed, -x_lstm).squeeze()
        
        mask = mask.view(-1)
        poss_loss = F.logsigmoid(poss_mm) * mask
        negs_loss = F.logsigmoid(negs_mm).mean(1) * mask
        
        total_loss = -(poss_loss + negs_loss)
        return total_loss.mean(), x_lstm

**简单测试**

In [52]:
corpus = Corpus(data_path, sort_by_len=False, uniform=False, freq_coef=0.75)
train_set = BobSueNegSampleDataSet(corpus.train_data, corpus.word_freqs)
model = NegSampleLM(len(corpus.vocab), n_embed=200)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=8,
    shuffle=True,
    collate_fn=neglm_collate_fn
)
inputs, poss, negs, lengths, mask = next(iter(train_loader))

loss, x_lstm = model(inputs, poss, negs, lengths, mask)
print('损失：', loss.item())

损失： 1.6722108125686646


* 定义一个辅助函数用来生成预测
* 这里采用LSTM后输出与输出部分的Embedding矩阵权重相乘，选取乘积最大的索引输出

In [58]:
def generate_prediction(model, x_lstm):
    with torch.no_grad():
        x_lstm = x_lstm.squeeze()  # (seq_len, embedding_size)
        embed_weight = model.embed_out.weight.transpose(0, 1)  # (embedding_size, n_words)
        preds = x_lstm @ embed_weight
        preds = preds.argmax(dim=-1)
    return preds

preds = generate_prediction(model, x_lstm)
preds = preds.view(-1, poss.shape[1])
print('Poss Shape:', poss.shape)
print('Preds Shape:', preds.shape)
((preds == poss) * mask).sum()

Poss Shape: torch.Size([8, 14])
Preds Shape: torch.Size([8, 14])


tensor(0.)

## 模型训练


### 定义基于负例采样的语言模型学习器

* 这里为了之后评估模型训练时间方便，统一将BatchSize固定为1，这样在模型计算过程中也不用进行padding了

In [76]:
class NegSampleLearner:
    
    def __init__(self, corpus, n_embed=200, dropout=0.5, n_negs=20,
                 batch_size=8):
        self.corpus = corpus
        self.model = NegSampleLM(len(corpus.vocab), n_embed, dropout).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters())
        self.n_negs = n_negs
        self.batch_size = batch_size
        
    def fit(self, num_epochs):        
        train_set = BobSueNegSampleDataSet(
            self.corpus.train_data,
            self.corpus.word_freqs,
            n_negs=self.n_negs,
        )
        valid_set = BobSueNegSampleDataSet(
            self.corpus.valid_data,
            self.corpus.word_freqs,
            n_negs=self.n_negs
        )
        
        train_loader = torch.utils.data.DataLoader(
            dataset=train_set,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=neglm_collate_fn
        )
        valid_loader = torch.utils.data.DataLoader(
            dataset=valid_set,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=neglm_collate_fn
        )
        
        
        for epoch in range(num_epochs):
            start_time = datetime.datetime.now()
            train_loss, train_words = self._make_train_step(train_loader)
            end_time = datetime.datetime.now()
            print(f'Epoch {epoch+1}:')
            print('Train Step --> Loss: {:.3f}, Words: {}, Time: {}s'.format(
                train_loss, train_words, (end_time-start_time).seconds
            ))
            
            valid_loss, valid_acc, valid_words = self._make_valid_step(valid_loader)
            print('Valid Step --> Loss: {:.3f}, Acc: {:.3f}, Words: {}'.format(
                valid_loss, valid_acc, valid_words
            ))
            
        
    def _make_train_step(self, train_loader):
        # 训练模式
        self.model.train()
        
        # 总损失
        total_loss = 0.0
        # 预测单词总数
        total_words = 0
        
        for inputs, poss, negs, lengths, mask in train_loader:
            inputs = inputs.to(device)
            poss = poss.to(device)
            negs = negs.to(device)
            lengths = lengths.to(device)
            mask = mask.to(device)
            
            # 模型损失
            loss, _ = self.model(inputs, poss, negs, lengths, mask)
            
            # 反向传播
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            # 统计信息
            sent_words = lengths.sum().item()
            total_words += sent_words
            total_loss += loss.item() * sent_words
        return total_loss / total_words, total_words
    
    def _make_valid_step(self, valid_loader):
        # 验证模式
        self.model.eval()
        
        # 总损失
        total_loss = 0.0
        # 预测正确个数，预测单词总数
        total_correct, total_words = 0, 0
        
        with torch.no_grad():
            for inputs, poss, negs, lengths, mask in valid_loader:
                inputs = inputs.to(device)
                poss = poss.to(device)
                negs = negs.to(device)
                lengths = lengths.to(device)
                mask = mask.to(device)
                
                # 模型损失
                loss, x_lstm = self.model(inputs, poss, negs, lengths, mask)
                
                # 生成预测，计算准确率
                preds = generate_prediction(self.model, x_lstm)
                preds = preds.view(-1, poss.shape[1])
                
                total_correct += ((preds == poss) * mask).sum().item()
                
                # 统计信息
                sent_words = lengths.sum().item()
                total_words += sent_words
                total_loss += loss.item() * sent_words
        return total_loss / total_words, total_correct / total_words, total_words
    
    def predict(self):
        test_set = BobSueNegSampleDataSet(
            self.corpus.test_data,
            self.corpus.word_freqs,
            n_negs=self.n_negs
        )
        
        test_loader = torch.utils.data.DataLoader(
            dataset=test_set,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=neglm_collate_fn
        )
        
        # 验证模式
        self.model.eval()

        # 预测正确个数，预测单词总数
        total_correct, total_words = 0, 0
        
        with torch.no_grad():
            for inputs, poss, negs, lengths, mask in test_loader:
                inputs = inputs.to(device)
                poss = poss.to(device)
                negs = negs.to(device)
                lengths = lengths.to(device)
                mask = mask.to(device)
                
                # 模型损失
                loss, x_lstm = self.model(inputs, poss, negs, lengths, mask)
                
                # 生成预测，计算准确率
                preds = generate_prediction(self.model, x_lstm)
                preds = preds.view(-1, poss.shape[1])
                
                total_correct += ((preds == poss) * mask).sum().item()
                
                # 统计信息
                sent_words = lengths.sum().item()
                total_words += sent_words
        return total_correct / total_words, total_words
        

### 模型训练

* 准备就绪，设定好参数开始训练

In [77]:
torch.cuda.empty_cache()
corpus = Corpus(data_path, sort_by_len=False, uniform=True, freq_coef=0.1)
learner = NegSampleLearner(corpus, n_embed=200, dropout=0.5, n_negs=20)
learner.fit(10)

Epoch 1:
Train Step --> Loss: 0.776, Words: 65331, Time: 4s
Valid Step --> Loss: 0.512, Acc: 0.193, Words: 7957
Epoch 2:
Train Step --> Loss: 0.489, Words: 65331, Time: 4s
Valid Step --> Loss: 0.455, Acc: 0.221, Words: 7957
Epoch 3:
Train Step --> Loss: 0.419, Words: 65331, Time: 4s
Valid Step --> Loss: 0.422, Acc: 0.239, Words: 7957
Epoch 4:
Train Step --> Loss: 0.376, Words: 65331, Time: 4s
Valid Step --> Loss: 0.409, Acc: 0.242, Words: 7957
Epoch 5:
Train Step --> Loss: 0.340, Words: 65331, Time: 4s
Valid Step --> Loss: 0.399, Acc: 0.234, Words: 7957
Epoch 6:
Train Step --> Loss: 0.315, Words: 65331, Time: 4s
Valid Step --> Loss: 0.396, Acc: 0.242, Words: 7957
Epoch 7:
Train Step --> Loss: 0.292, Words: 65331, Time: 4s
Valid Step --> Loss: 0.399, Acc: 0.245, Words: 7957
Epoch 8:
Train Step --> Loss: 0.274, Words: 65331, Time: 4s
Valid Step --> Loss: 0.402, Acc: 0.254, Words: 7957
Epoch 9:
Train Step --> Loss: 0.259, Words: 65331, Time: 4s
Valid Step --> Loss: 0.410, Acc: 0.244, Word

* 这里简单的看一下训练10个Epoch后的测试集结果

In [78]:
test_acc, test_word = learner.predict()
print('测试集预测总词数：', test_word)
print('测试集预测准确率：', test_acc)

测试集预测总词数： 8059
测试集预测准确率： 0.25139595483310584


# 使用上下文信息Context的语言模型

* 这部分的内容基于Part1的实现，不过这次采用的上文信息
* 我们需要重新构建我们的数据集，并将上文信息输入到模型中用于提高准确率

## 数据处理

### 定义语料库

* 这里我们在Part1的基础上将上下文句子按照制表符划分保存到对应的data列表里

In [79]:
class ContextCorpus:
    
    def __init__(self, data_path):
        self.vocab = Vocab(data_path / 'bobsue.voc.txt')
        self.train_data = self.tokenize(data_path / 'bobsue.prevsent.train.tsv')
        self.valid_data = self.tokenize(data_path / 'bobsue.prevsent.dev.tsv')
        self.test_data = self.tokenize(data_path / 'bobsue.prevsent.test.tsv')
    
    def tokenize(self, text_path):
        with open(text_path) as f:
            index_data = []
            for s in f.readlines():
                t = s.split('\t')
                index_data.append(
                    (self.sentence_to_index(t[0]),
                     self.sentence_to_index(t[1]))
                )
            return index_data
                
    def sentence_to_index(self, s):
        return [self.vocab.stoi[w] for w in s.split()]
    
    def index_to_sentence(self, x):
        return ' '.join([self.vocab.itos[i] for i in x])                

In [80]:
corpus = ContextCorpus(data_path)
print('训练集句子数目：', len(corpus.train_data))
print('验证集句子数目：', len(corpus.valid_data))
print('测试集句子数目：', len(corpus.test_data))
print('-' * 60)

print('数据样本：')
for i in range(5):
    print(corpus.train_data[i][0])
    print(corpus.train_data[i][1])
    print(corpus.index_to_sentence(corpus.train_data[i][0]))
    print(corpus.index_to_sentence(corpus.train_data[i][1]))

训练集句子数目： 6036
验证集句子数目： 750
测试集句子数目： 750
------------------------------------------------------------
数据样本：
[0, 7, 157, 17, 6, 103, 275, 2, 1]
[0, 16, 235, 372, 10, 60, 3, 75, 618, 39, 2, 1]
<s> Sue realized she was really bored . </s>
<s> She ate quickly and asked to be taken home . </s>
[0, 4, 211, 19, 505, 31, 8, 192, 2, 1]
[0, 38, 192, 222, 32, 31, 4, 2, 1]
<s> Bob fell in love with a girl . </s>
<s> The girl broke up with Bob . </s>
[0, 586, 9, 51, 65, 9, 17, 170, 23, 215, 227, 2, 1]
[0, 7, 842, 2, 1]
<s> Eventually her friend told her she wasn 't having fun . </s>
<s> Sue apologized . </s>
[0, 38, 518, 6, 123, 363, 226, 4, 2, 1]
[0, 12, 150, 18, 8, 261, 3, 546, 102, 5, 1097, 2, 1]
<s> The company was called pizza man Bob . </s>
<s> He tried for a year to break into the market . </s>
[0, 4, 6, 757, 25, 8, 37, 42, 2, 1]
[0, 200, 706, 14, 5, 26, 15, 427, 228, 2, 1]
<s> Bob was starting at a new school . </s>
<s> So far , the day had gone well . </s>


### 定义上下文语言模型的DataSet

* `DateSet`中包含了每句话的上文信息、输入信息和预测目标信息

In [81]:
class BobSueContextDataset(torch.utils.data.Dataset):
    
    def __init__(self, index_data):
        self.index_data = index_data
        
    def __getitem__(self, i):
        contexts = torch.LongTensor(self.index_data[i][0])
        inputs = torch.LongTensor(self.index_data[i][1][:-1])
        targets = torch.LongTensor(self.index_data[i][1][1:])
        return contexts, inputs, targets
        
    def __len__(self):
        return len(self.index_data)

In [82]:
train_set = BobSueContextDataset(corpus.train_data)
print('训练集大小：', len(train_set))
print('训练集样本：')
contexts, inputs, targets = train_set[10]
print('\t上文：', list(contexts.numpy()))
print('\t', corpus.index_to_sentence(contexts.numpy()))
print('\t输入：', list(inputs.numpy()))
print('\t', corpus.index_to_sentence(inputs.numpy()))
print('\t输出：', list(targets.numpy()))
print('\t', corpus.index_to_sentence(targets.numpy()))

训练集大小： 6036
训练集样本：
	上文： [0, 16, 1303, 5, 709, 507, 3, 11, 1240, 107, 2, 1]
	 <s> She followed the cute guy to his science class . </s>
	输入： [0, 7, 208, 601, 28, 10, 1276, 25, 5, 709, 507, 2]
	 <s> Sue sat behind him and stared at the cute guy .
	输出： [7, 208, 601, 28, 10, 1276, 25, 5, 709, 507, 2, 1]
	 Sue sat behind him and stared at the cute guy . </s>


### 定义上下文语言模型的DataLoader

* 与Part1相同，这里我们也通过自定义collate_fn来处理

In [83]:
def ctx_collate_fn(batch):
    # 首先将batch的格式进行转换
    # batch[0]：Contexts
    # batch[1]: Inputs
    # batch[2]: Targets
    batch = list(zip(*batch))
    
    ctx_lengths = torch.LongTensor([len(x) for x in batch[0]])
    inp_lengths = torch.LongTensor([len(x) for x in batch[1]])
    
    contexts = nn.utils.rnn.pad_sequence(batch[0], batch_first=True)
    inputs = nn.utils.rnn.pad_sequence(batch[1], batch_first=True)
    targets = nn.utils.rnn.pad_sequence(batch[2], batch_first=True)

    mask = (targets != 0).float()
    
    return contexts, inputs, targets, ctx_lengths, inp_lengths, mask

**简单测试**

In [84]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=8,
    shuffle=True,
    collate_fn=ctx_collate_fn
)
contexts, inputs, targets, ctx_lengths, inp_lengths, mask = next(iter(train_loader))
print('Contexts Shape:', contexts.shape)
print('Inputs Shape:', inputs.shape)
print('Targets Shape:', targets.shape)
print('-' * 60)
print('Contexts Lengths：')
print(ctx_lengths)
print('Inputs Lengths: ')
print(inp_lengths)
print('-' * 60)
print('Mask：')
print(mask)

Contexts Shape: torch.Size([8, 14])
Inputs Shape: torch.Size([8, 16])
Targets Shape: torch.Size([8, 16])
------------------------------------------------------------
Contexts Lengths：
tensor([ 9, 10, 11,  9, 13, 14,  7, 13])
Inputs Lengths: 
tensor([11, 12, 12, 10, 12, 16,  9, 10])
------------------------------------------------------------
Mask：
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.]])


## 定义网络结构

### 定义模型

* 和Part1类似定义一个LSTM的网络架构
* 与Part1不同的是，这次我们先将context信息传入第一个LSTM，之后将最后一个hidden作为第二个LSTM的初始值，这样进行训练

In [87]:
class ContextLM(nn.Module):
    
    def __init__(self, n_words, n_embed=200, n_hidden=200, dropout=0.5):
        super(ContextLM, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.embed = nn.Embedding(n_words, n_embed)
        self.encoder = nn.LSTM(n_embed, n_hidden, batch_first=True)
        self.decoder = nn.LSTM(n_embed, n_hidden, batch_first=True)
        self.linear = nn.Linear(n_hidden, n_words)
        
    def forward(self, contexts, inputs, ctx_lengths, inp_lengths):
        # 对上一句话进行编码
        ctx_emb = self.drop(self.embed(contexts))
        ctx_emb = nn.utils.rnn.pack_padded_sequence(
            ctx_emb, ctx_lengths, 
            batch_first=True, 
            enforce_sorted=False
        )
        _, (h_n, c_n) = self.encoder(ctx_emb)
        
        # 对当前句子进行预测
        inp_emb = self.drop(self.embed(inputs))
        inp_emb = nn.utils.rnn.pack_padded_sequence(
            inp_emb, inp_lengths,
            batch_first=True,
            enforce_sorted=False
        )
        inp_out, _ = self.decoder(inp_emb, (h_n, c_n))
        inp_out, _ = nn.utils.rnn.pad_packed_sequence(inp_out, batch_first=True)
            
        return self.linear(self.drop(inp_out))
        
model = ContextLM(len(corpus.vocab), 200, 200)

**简单测试**

In [88]:
outputs = model(contexts, inputs, ctx_lengths, inp_lengths)
print('模型输出Shape：', outputs.shape)

模型输出Shape： torch.Size([8, 16, 1498])


### 定义损失函数

In [89]:
class MaskCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(MaskCrossEntropyLoss, self).__init__()
        self.celoss = nn.CrossEntropyLoss(reduction='none')
    
    def forward(self, outputs, targets, mask):
        # outputs shape: (batch_size * max_len, vocab_size)
        outputs = outputs.view(-1, outputs.size(2))
        # targets shape: (batch_size * max_len)
        targets = targets.view(-1)
        # mask shape: (batch_size * max_len)
        mask = mask.view(-1)
        loss = self.celoss(outputs, targets) * mask
        return torch.sum(loss) / torch.sum(mask)

**简单测试**

In [90]:
criterion = MaskCrossEntropyLoss()
loss = criterion(outputs, targets, mask)
print('Loss:', loss.item())

Loss: 7.322925090789795


## 模型训练与预测

### 定义基于上文信息的语言模型学习器

* 这里为了方便起见和上一问相同，每一个批量就单独一句话

In [103]:
class ContextLearner:
    
    def __init__(self, corpus, n_embed=200, n_hidden=200, dropout=0.5,
                 batch_size=128, early_stopping_round=5):
        self.corpus = corpus
        self.model = ContextLM(len(corpus.vocab), n_embed, n_hidden, dropout)
        self.model.to(device)
        self.criterion = MaskCrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters())
        self.history = defaultdict(list)
        self.early_stopping_round = early_stopping_round
        self.batch_size = batch_size
    
    def fit(self, num_epoch):
        train_set = BobSueContextDataset(
            self.corpus.train_data
        )
        train_loader = torch.utils.data.DataLoader(
            dataset=train_set,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=ctx_collate_fn
        )
        
        valid_set = BobSueContextDataset(
            self.corpus.valid_data
        )
        valid_loader = torch.utils.data.DataLoader(
            dataset=valid_set,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=ctx_collate_fn
        )
        
        no_improve_round = 0
        
        for epoch in range(num_epoch):
            train_loss, train_acc, train_words = self._make_train_step(train_loader)
            print(f'Epoch {epoch+1}:')
            print('Train Step --> Loss: {:.3f}, Acc: {:.3f}, Words: {}'.format(
                train_loss, train_acc, train_words))
            
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            
            valid_loss, valid_acc, valid_words = self._make_valid_step(valid_loader)
            print('Valid Step --> Loss: {:.3f}, Acc: {:.3f}, Words: {}'.format(
                valid_loss, valid_acc, valid_words))
            
            self.history['valid_loss'].append(valid_loss)
            self.history['valid_acc'].append(valid_acc)
            
            # 根据验证集的准确率进行EarlyStopping
            if self.history['valid_acc'][-1] < max(self.history['valid_acc']):
                no_improve_round += 1
            else:
                no_improve_round = 0
            if no_improve_round == self.early_stopping_round:
                print(f'Early Stopping at Epoch {epoch+1}')
                break
            
            
    def _make_train_step(self, train_loader):
        self.model.train()
        
        total_loss = 0.0
        total_correct, total_words = 0, 0
        
        for batch in train_loader:
            contexts = batch[0].to(device)
            inputs = batch[1].to(device)
            targets = batch[2].to(device)
            ctx_lengths = batch[3].to(device)
            inp_lengths = batch[4].to(device)
            mask = batch[5].to(device)
            
            outputs = self.model(contexts, inputs, ctx_lengths, inp_lengths)
            loss = self.criterion(outputs, targets, mask)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            total_correct += (outputs.argmax(-1) == targets).sum().item()
            total_words += torch.sum(inp_lengths).item()
            total_loss += loss.item() * torch.sum(mask).item()
        
        return total_loss / total_words, total_correct / total_words, total_words
    
    def _make_valid_step(self, valid_loader):
        self.model.eval()
        
        total_loss = 0.0
        total_correct, total_words = 0, 0
        
        with torch.no_grad():
            for batch in valid_loader:
                contexts = batch[0].to(device)
                inputs = batch[1].to(device)
                targets = batch[2].to(device)
                ctx_lengths = batch[3].to(device)
                inp_lengths = batch[4].to(device)
                mask = batch[5].to(device)

                outputs = self.model(contexts, inputs, ctx_lengths, inp_lengths)
                loss = self.criterion(outputs, targets, mask)

                total_correct += (outputs.argmax(-1) == targets).sum().item()
                total_words += torch.sum(inp_lengths).item()
                total_loss += loss.item() * torch.sum(mask).item()
        
        return total_loss / total_words, total_correct / total_words, total_words
    
    def predict(self):
        test_set = BobSueContextDataset(
            self.corpus.test_data
        )
        test_loader = torch.utils.data.DataLoader(
            dataset=test_set,
            batch_size=1,
            shuffle=False,
            collate_fn=ctx_collate_fn
        )
        
        self.model.eval()
        
        total_loss = 0.0
        total_correct, total_words = 0, 0
        test_result = defaultdict(list)
        
        with torch.no_grad():
            for batch in test_loader:
                contexts = batch[0].to(device)
                inputs = batch[1].to(device)
                targets = batch[2].to(device)
                ctx_lengths = batch[3].to(device)
                inp_lengths = batch[4].to(device)
                mask = batch[5].to(device)

                outputs = self.model(contexts, inputs, ctx_lengths, inp_lengths)
                loss = self.criterion(outputs, targets, mask)

                total_correct += (outputs.argmax(-1) == targets).sum().item()
                total_words += torch.sum(inp_lengths).item()
                total_loss += loss.item() * torch.sum(mask).item()
        
                test_result['preds'].append(outputs.argmax(-1).data.cpu().numpy()[0])
                test_result['targets'].append(targets.data.cpu().numpy()[0])
            
        return total_loss / total_words, total_correct / total_words, total_words, test_result

### 模型训练

* 这里设定好参数就可以开始训练了

In [104]:
torch.cuda.empty_cache()
corpus = ContextCorpus(data_path)
learner = ContextLearner(corpus, n_embed=200, n_hidden=200, dropout=0.5)
learner.fit(1000)

Epoch 1:
Train Step --> Loss: 5.705, Acc: 0.161, Words: 65331
Valid Step --> Loss: 4.661, Acc: 0.229, Words: 7957
Epoch 2:
Train Step --> Loss: 4.636, Acc: 0.227, Words: 65331
Valid Step --> Loss: 4.372, Acc: 0.243, Words: 7957
Epoch 3:
Train Step --> Loss: 4.392, Acc: 0.243, Words: 65331
Valid Step --> Loss: 4.157, Acc: 0.260, Words: 7957
Epoch 4:
Train Step --> Loss: 4.201, Acc: 0.256, Words: 65331
Valid Step --> Loss: 3.990, Acc: 0.287, Words: 7957
Epoch 5:
Train Step --> Loss: 4.056, Acc: 0.272, Words: 65331
Valid Step --> Loss: 3.871, Acc: 0.296, Words: 7957
Epoch 6:
Train Step --> Loss: 3.942, Acc: 0.283, Words: 65331
Valid Step --> Loss: 3.776, Acc: 0.304, Words: 7957
Epoch 7:
Train Step --> Loss: 3.849, Acc: 0.296, Words: 65331
Valid Step --> Loss: 3.698, Acc: 0.325, Words: 7957
Epoch 8:
Train Step --> Loss: 3.775, Acc: 0.307, Words: 65331
Valid Step --> Loss: 3.636, Acc: 0.330, Words: 7957
Epoch 9:
Train Step --> Loss: 3.714, Acc: 0.313, Words: 65331
Valid Step --> Loss: 3.590

### 模型测试

* 通过上面的训练，我们看一下测试集上的表现

In [105]:
test_loss, test_acc, test_words, test_result = learner.predict()
print('测试集上的结果 --> Loss: {:.3f}, Acc: {:.3f}, Words: {}'.format(
    test_loss, test_acc, test_words))

测试集上的结果 --> Loss: 3.329, Acc: 0.367, Words: 8059


可以看到我们在测试集上的表现已经超越了Part1当中的情况，在Part1中我们的准确率是0.341

In [106]:
print('预测句子数量：', len(test_result['preds']))
print('-' * 60)

sample_index = 4
print('结果样例：')
print('预测值\t', test_result['preds'][sample_index])
print('实际值\t', test_result['targets'][sample_index])
print('预测句子\t', corpus.index_to_sentence(test_result['preds'][sample_index]))
print('实际句子\t', corpus.index_to_sentence(test_result['targets'][sample_index]))

预测句子数量： 750
------------------------------------------------------------
结果样例：
预测值	 [ 12   6   8   6  23 394  84   1]
实际值	 [ 12 225  92 170  23 134   2   1]
预测句子	 He was a was 't sure money </s>
实际句子	 He noticed there wasn 't much . </s>


## 错误分析

* 这里我们统计了一下常见的35个预测错误（实际值，预测值）

In [107]:
mistake_counter = Counter()
for i in range(len(test_result['targets'])):
    for j in range(len(test_result['targets'][i])):
        pred, target = test_result['preds'][i][j], test_result['targets'][i][j]
        if pred != target:
            pred, target = corpus.vocab.itos[pred], corpus.vocab.itos[target]
            mistake_counter[(target, pred)] += 1
mistake_counter.most_common(35)

[(('to', '.'), 50),
 (('had', 'was'), 49),
 (('decided', 'was'), 42),
 (('and', '.'), 35),
 (('his', 'the'), 35),
 (('her', 'the'), 34),
 (('for', '.'), 33),
 (('.', 'to'), 32),
 (('Bob', 'He'), 31),
 (('in', '.'), 26),
 (('Sue', 'Bob'), 25),
 (('a', 'the'), 24),
 (('His', 'He'), 23),
 (('the', 'a'), 23),
 (('got', 'was'), 22),
 (('a', 'to'), 22),
 (('went', 'was'), 21),
 (('he', 'to'), 21),
 (('Sue', 'She'), 20),
 (('.', 'and'), 19),
 ((',', '.'), 19),
 (('Her', 'She'), 19),
 (('!', '.'), 18),
 (('at', '.'), 17),
 (("'s", 'was'), 17),
 (('wanted', 'was'), 17),
 (('and', 'to'), 17),
 (('for', 'to'), 16),
 (('the', '.'), 16),
 (('asked', 'was'), 16),
 (('.', 'the'), 15),
 (('didn', 'was'), 15),
 (('it', 'her'), 14),
 (('a', '.'), 14),
 (('on', '.'), 14)]

* 这里我们可以看出，在Part1当中主要的句首错误已经得到了有效的缓解，但是对于介词错误和Be动词错误模型还是不能很好的预测
* 这主要是由于后两类错误在没有后文信息的情况下很难作出判断